In [5]:
from genfft import generate_fft, default_config, fft_two_factor_twiddle
import numpy as np
from copy import copy

In [6]:
cfg = copy(default_config)
cfg["generator_path"] = "../genfft"
cfg["compile_args"] = ["-x", "c++", "-O3", "-include", "genfft/codelet.hh", "-shared"]

In [10]:
fft12 = fft_two_factor_twiddle(cfg, 3, 4)

x = np.arange(12, dtype='complex64')

fft12(x) - np.fft.fft(x)

In [13]:
fft27 = fft_two_factor_twiddle(cfg, 3, 9)
x = np.arange(27, dtype='complex64')
np.abs(fft27(x) - np.fft.fft(x)).max()

8.962143078707292e-06

In [14]:
"onuth".splitlines()

['onuth']

In [17]:
x = np.arange(10, dtype='complex64')

In [18]:
x.real

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=float32)

In [19]:
x.real[3] = 42

In [20]:
x

array([ 0.+0.j,  1.+0.j,  2.+0.j, 42.+0.j,  4.+0.j,  5.+0.j,  6.+0.j,
        7.+0.j,  8.+0.j,  9.+0.j], dtype=complex64)